In [5]:
import random
import numpy as np


class Board:
    def __init__(self, n):
        self.n_queen = n
        self.map = [[0 for j in range(n)] for i in range(n)]
        self.encoded = self.encode()
        self.fit = 0

    def set_queens(self):
        for i in range(self.n_queen):
            j = random.randint(0, self.n_queen - 1)
            self.map[i][j] = 1

    def fitness(self):
        self.fit = self.n_queen * (self.n_queen - 1) // 2
        for i in range(self.n_queen):
            for j in range(self.n_queen):
                if self.map[i][j] == 1:
                    for k in range(1, self.n_queen - i):
                        if self.map[i + k][j] == 1:
                            self.fit -= 1
                        if j - k >= 0 and self.map[i + k][j - k] == 1:
                            self.fit -= 1
                        if j + k < self.n_queen and self.map[i + k][j + k] == 1:
                            self.fit -= 1
                            
    def encode(self):
        s = ''
        for row in self.map:
            for i in range(len(row)):
                if row[i] == 1:
                    s += str(i)
                    break
        return s
    
    
    def decode_to_board(self, enc):
        self.map = [[0 for j in range(n)] for i in range(n)]
        c = 0
        for s in enc:
            self.map[c][s] = 1
            c += 1
            
        self.encoded = enc

    
    def show(self):
        print(np.matrix(self.map))
        print("Fitness: ",  self.fit)

        
if __name__ == '__main__':
    test = Board(5)
    test.set_queens()
    test.fitness()
    test.show()
    print(test.encode())

[[0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]
Fitness:  4
10122


In [6]:
class Hill_Climb:
    
    def __init__(self, n):
        self.step = 0
        self.map = n
        self.q_pos = set()  
    
    def moveQueen(self):
        temp = self.map.map[0]
        self.map.map[0] = [0,0,0,0,0]
        mfit = 0
        mindx = 0
        for i in range(5):
            self.map.map[i] = [0,0,0,0,0]
            mfit = 0
            mindx = 0
            for j in range(5):
                self.map.map[i][j] = 1
                self.map.fitness()
                if(self.map.fit > mfit):
                    mfit = self.map.fit
                    mindx = j
                self.map.map[i][j] = 0
                if(mfit == 10):
                    print("Finished")
                    self.map.fitness()
                    self.map.show()
                    return self
            self.map.map[i][mindx] = 1
        self.moveQueen()
        
if __name__ == '__main__':
    home = Board(5)
    home.set_queens()
    home.fitness()
    home.show()
    print("\n")
    climb = Hill_Climb(home)
    climb.moveQueen()
        
    

[[0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]]
Fitness:  4


Finished
[[1 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]]
Fitness:  10


In [22]:
class Genetic:
    def __init__(self, n):
        self.step = 0
        self.states = dict()
        self.changed_states = []

        for i in range(4):
            board = Board(n)
            board.set_queens()
            board.fitness()
            self.states[board.encode()] = board.fit
        
        
    # Sets self.changed_states to selected genes 
    def selection(self):
        total_fit = 0
        for r in self.states:
            total_fit += self.states[r]
        
        # sorting by fitness
        self.states = {k : v for k, v, in sorted(self.states.items(), key=lambda item: item[1], reverse=True)}
        keys = list(self.states.keys())
        
        print('States', self.states)
        selects = dict()
        # calculating selection probabilities
        for r in keys:
            selects[r] = (self.states[r] / total_fit)
        
        # selecting states
        for i in range(4):
            r = np.random.randint(0, 101) / 100
             
            if r < selects[keys[0]]:
                self.changed_states.append(keys[0])
            
            elif r >= selects[keys[0]] and r < selects[keys[1]]:
                self.changed_states.append(keys[1])
                
            elif r >= selects[keys[1]] and r < selects[keys[2]]:
                self.changed_states.append(keys[2])
        
            else:
                self.changed_states.append(keys[3])
        
        print('Selections', self.changed_states)
        
                
    
    # Updates self.changed_states after crossover
    def crossover(self):

        r1 = np.random.randint(0, len(self.changed_states[0])-1)
        new_0 = self.changed_states[0][:r1] + self.changed_states[1][r1:]
        new_1 = self.changed_states[1][:r1] + self.changed_states[0][r1:]
        self.changed_states[0] = new_0
        self.changed_states[1] = new_1
        
        r2 = np.random.randint(0, len(self.changed_states[0])-1)
        new_2 = self.changed_states[2][:r1] + self.changed_states[3][r1:]
        new_3 = self.changed_states[3][:r1] + self.changed_states[2][r1:]
        self.changed_states[2] = new_2
        self.changed_states[3] = new_3

        print('Crossover', self.changed_states)
    

    # Updates self.changed_states after mutation
    def mutate(self):
        pass
                
        
        
if __name__ == '__main__':
    genetic = Genetic(8)
    genetic.selection()
    genetic.crossover()

States {'63234053': 20, '75511065': 20, '50103614': 19, '35456065': 19}
Selections ['35456065', '35456065', '35456065', '63234053']
Crossover ['35456065', '35456065', '35456053', '63234065']
